*Note that this notebook should be run with a dedicated compute cluster with the ML Runtime.  It was developed using DBR 16.2 ML originally.* 

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
%pip install git+https://github.com/databricks-industry-solutions/redox-ehr-api

In [0]:
%restart_python

## Standard Usage 
***

In [0]:
databricks_secret_scope = "redox-field-eng"

redox_private_key = dbutils.secrets.get(scope = databricks_secret_scope, key = "redox_private_key")
redox_client_id = dbutils.secrets.get(scope = databricks_secret_scope, key = "redox_client_id")
redox_source_id = dbutils.secrets.get(scope = databricks_secret_scope, key = "redox_source_id")
redox_public_kid = dbutils.secrets.get(scope = databricks_secret_scope, key = "redox_public_kid")

print(f""" 
      redox_private_key: {redox_private_key}
      redox_client_id: {redox_client_id}
      redox_source_id: {redox_source_id}
      redox_public_kid: {redox_public_kid}
""")

In [0]:
import json

redox_auth_json = f"""
{{
  "kty": "RSA",
  "kid": "{redox_public_kid}",
  "alg": "RS384",
  "use": "sig"
}}
"""

json.loads(redox_auth_json)

In [0]:
from redoxwrite.auth import * 
from redoxwrite.endpoint import *

In [0]:
auth = RedoxApiAuth(
  redox_client_id
  ,redox_private_key
  ,redox_auth_json
  ,redox_source_id
)
print("Is connection successful? " + str(auth.can_connect()))

In [0]:
#All Redox FHIR request URLs start with this base: https://api.redoxengine.com/fhir/R4/[organization-name]/[environment-type]/
redox_base_url = 'https://api.redoxengine.com/fhir/R4/redox-fhir-sandbox/Development/'

rapi = RedoxApiRequest(auth, base_url = redox_base_url)

In [0]:
#creating an observation for remaining length of 4 day stay at a hospital 
observation = """
{
   "resourceType":"Bundle",
   "entry":[
      {
         "resource":{
            "category":[
               {
                  "coding":[
                     {
                        "code":"survey",
                        "display":"Survey",
                        "system":"http://terminology.hl7.org/CodeSystem/observation-category"
                     }
                  ]
               }
            ],
            "code":{
               "coding":[
                  {
                     "code":"78033-8",
                     "display":"Remaining Hospital Stay",
                     "system":"http://loinc.org"
                  }
               ],
               "text":"Remaining Hospital Stay"
            },
            "effectiveDateTime":"2024-01-28T18:06:33.245-05:00",
            "issued":"2024-01-28T18:06:33.245-05:00",
            "resourceType":"Observation",
            "status":"final",
            "valueQuantity":{
               "code":"days",
               "system":"https://www.nubc.org/CodeSystem/RevenueCodes",
               "unit":"days",
               "value":4
            },
            "subject": {
              "reference": "Patient/58117110-ae47-452a-be2c-2d82b3a9e24b"
            },
            "identifier": [
            {
              "system": "urn:databricks",
              "value": "1234567890"
            }
          ]
         }
      },
      {
         "resource":{
           "resourceType": "Patient",
           "identifier": [
            {
              "system": "urn:redox:health-one:MR",
              "value": "0000991458"
            },
            {
              "system": "http://hl7.org/fhir/sid/us-ssn",
              "value": "547-01-9991"
            }
          ]
         }
      }
   ]
}
"""

In [0]:
result = rapi.make_request(
  http_method="post"
  ,resource="Observation"
  ,action="$observation-create"
  ,data=observation
)

In [0]:
import json

print(f"Response Status Code: {result['response']['response_status_code']}")
print("\n")
print(json.dumps(json.loads(result['response']['response_text']), indent=4))

In [0]:
import mlflow
# import base64
# from databricks.sdk import WorkspaceClient
from os import environ

In [0]:
# from databricks.sdk import WorkspaceClient
# w = WorkspaceClient()
# base64.b64decode(w.secrets.get_secret(scope = databricks_secret_scope, key = "redox_private_key").value).decode('utf-8')

In [0]:
environ['REDOX_PRIVATE_KEY'] = redox_private_key
environ['REDOX_CLIENT_ID'] = redox_client_id
environ['REDOX_SOURCE_ID'] = redox_source_id
environ['REDOX_PUBLIC_KID'] = redox_public_kid

In [0]:
class RedoxMakeRequest(mlflow.pyfunc.PythonModel):
    def __init__(self, base_url, redox_secret_scope):
        self.base_url = base_url
        self.redox_secret_scope = redox_secret_scope

    def load_context(self, context):
        import redoxwrite.auth
        import redoxwrite.endpoint
        # from base64 import b64decode
        # from databricks.sdk import WorkspaceClient
        # w = WorkspaceClient()
        from os import environ
        self.redox_private_key = environ['REDOX_PRIVATE_KEY']
        self.redox_client_id = environ['REDOX_CLIENT_ID']
        self.redox_source_id = environ['REDOX_SOURCE_ID']
        self.redox_public_kid = environ['REDOX_PUBLIC_KID']
        self.redox_auth_json = f"""
                {{
                "kty": "RSA",
                "kid": "{self.redox_public_kid}",
                "alg": "RS384",
                "use": "sig"
                }}
            """
        self.auth = RedoxApiAuth(
                self.redox_client_id
                ,self.redox_private_key
                ,self.redox_auth_json
                ,self.redox_source_id
            )
        self.rapi = RedoxApiRequest(self.auth, base_url = self.base_url)
    
    def predict(self, context, model_input, params = None): 
        results = []
        # Apply the self.rapi.make_request function to each row of the DataFrame
        # Assumes that the model input contains the following columns:
            # http_method, resource, action, data where data is a value FHIR payload
        # Convert Pandas DF to string.
        for row in model_input.itertuples(index=False):
            result = self.rapi.make_request(
                http_method=row.http_method
                ,resource=row.resource
                ,action=row.action
                ,data=row.data
            )
            respone_text = result['response']['response_text']
            results.append(respone_text)
        return results

In [0]:
import pandas as pd

df = pd.DataFrame({"http_method": ["post", "post"], "resource": ["Observation", "DiagnosticReport"], "action": ["$observation-create", "_search"], "data": [observation, "subject=Patient/81c2f5eb-f99f-40c4-b504-59483e6148d7'"]})
df

In [0]:
signature = mlflow.models.infer_signature(df, ["redox_make_request"])
signature

In [0]:
conda_env = mlflow.pyfunc.get_default_conda_env()
conda_env["dependencies"][-1]["pip"].append("git+https://github.com/databricks-industry-solutions/redox-ehr-api")
# conda_env["dependencies"][-1]["pip"].append("databricks-sdk")
conda_env["dependencies"][-1]["pip"].append("pyjwt")
conda_env["dependencies"][-1]["pip"].append("cryptography")
print(conda_env)

In [0]:
with mlflow.start_run():

    # Ensure the standard python libraries and the redox-ehr-api classed and methods are installed in the conda environment
    conda_env = mlflow.pyfunc.get_default_conda_env()
    conda_env["dependencies"][-1]["pip"].append("git+https://github.com/databricks-industry-solutions/redox-ehr-api")
    # conda_env["dependencies"][-1]["pip"].append("databricks-sdk")
    conda_env["dependencies"][-1]["pip"].append("pyjwt")
    conda_env["dependencies"][-1]["pip"].append("cryptography")
    
    mlflow.pyfunc.log_model(
        artifact_path="model"
        ,python_model=RedoxMakeRequest(
            base_url=redox_base_url
            ,redox_secret_scope=databricks_secret_scope
        )
        ,signature=signature
        ,conda_env=conda_env
    )
 
    run_id = mlflow.active_run().info.run_id

In [0]:
logged_model = f"runs:/{run_id}/model"

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
loaded_model.predict(df)

In [0]:
catalog = "redox"
schema = "himss25"
model_name = "redox_make_request"
full_model_name = f"{catalog}.{schema}.{model_name}"

mlflow.set_registry_uri("databricks-uc")

mlflow.register_model(f"runs:/{run_id}/model", full_model_name)

In [0]:
client = mlflow.MlflowClient()

# Search for all versions of the model
model_version_infos = client.search_model_versions(f"name = '{full_model_name}'")

# Find the latest version
latest_version = max([model_version_info.version for model_version_info in model_version_infos])
print(f"""
    The latest version of the model {full_model_name} in Unity Catalog is version {latest_version}.
""")

In [0]:
prior_model_version = client.get_model_version_by_alias(full_model_name, "latest_version")

if prior_model_version.version != latest_version:
    client.delete_registered_model_alias(full_model_name, "latest_version")

# Create an alias for the model version
client.set_registered_model_alias(full_model_name, "latest_version", latest_version)

In [0]:
# Load Unity Catalog model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(f"models:/{full_model_name}@latest_version")

# Predict on a Pandas DataFrame.
loaded_model.predict(df)

Mnaually serve the model from Unity Catalog and Set Required Variables

Browser Version:

{"dataframe_records":[{"http_method":"post","resource":"Observation","action":"$observation-create","data":"\n{\n   \"resourceType\":\"Bundle\",\n   \"entry\":[\n      {\n         \"resource\":{\n            \"category\":[\n               {\n                  \"coding\":[\n                     {\n                        \"code\":\"survey\",\n                        \"display\":\"Survey\",\n                        \"system\":\"http://terminology.hl7.org/CodeSystem/observation-category\"\n                     }\n                  ]\n               }\n            ],\n            \"code\":{\n               \"coding\":[\n                  {\n                     \"code\":\"78033-8\",\n                     \"display\":\"Remaining Hospital Stay\",\n                     \"system\":\"http://loinc.org\"\n                  }\n               ],\n               \"text\":\"Remaining Hospital Stay\"\n            },\n            \"effectiveDateTime\":\"2024-01-28T18:06:33.245-05:00\",\n            \"issued\":\"2024-01-28T18:06:33.245-05:00\",\n            \"resourceType\":\"Observation\",\n            \"status\":\"final\",\n            \"valueQuantity\":{\n               \"code\":\"days\",\n               \"system\":\"https://www.nubc.org/CodeSystem/RevenueCodes\",\n               \"unit\":\"days\",\n               \"value\":4\n            },\n            \"subject\": {\n              \"reference\": \"Patient/58117110-ae47-452a-be2c-2d82b3a9e24b\"\n            },\n            \"identifier\": [\n            {\n              \"system\": \"urn:databricks\",\n              \"value\": \"1234567890\"\n            }\n          ]\n         }\n      },\n      {\n         \"resource\":{\n           \"resourceType\": \"Patient\",\n           \"identifier\": [\n            {\n              \"system\": \"urn:redox:health-one:MR\",\n              \"value\": \"0000991458\"\n            },\n            {\n              \"system\": \"http://hl7.org/fhir/sid/us-ssn\",\n              \"value\": \"547-01-9991\"\n            }\n          ]\n         }\n      }\n   ]\n}\n"}]}